## Download HF dataset

In [ ]:
import pandas as pd

n_per_dataset = 200
df = pd.DataFrame(columns=['prompt_en', 'prompt_ar', 'response_en', 'response_ar', 'source'])

In [ ]:
from datasets import load_dataset

dataset_repo = 'akoksal/LongForm'
dataset = load_dataset(dataset_repo, split='test')

df_longForm = pd.DataFrame(dataset)
df_longForm.shape

In [ ]:
df_longForm.head()

In [ ]:
subsampled_df = df_longForm[['input', 'output']].sample(n=n_per_dataset).reset_index(drop=True)

df = pd.concat([df, pd.DataFrame(
    {'prompt_en': subsampled_df['input'].tolist(),
     'prompt_ar': '',
     'response_en': subsampled_df['output'].tolist(),
     'response_ar': '',
     'source': dataset_repo,
    })], ignore_index=True)

df.shape

In [ ]:
from datasets import load_dataset

dataset_repo = 'mlabonne/alpagasus'
dataset = load_dataset(dataset_repo, split='train')

df_alpagasus = pd.DataFrame(dataset)
df_alpagasus.shape

In [ ]:
df_alpagasus = df_alpagasus[df_alpagasus['input'] == '']
df_alpagasus.shape

In [ ]:
df_alpagasus.head()

In [ ]:
subsampled_df = df_alpagasus[['instruction', 'output']].sample(n=n_per_dataset).reset_index(drop=True)

df = pd.concat([df, pd.DataFrame(
    {'prompt_en': subsampled_df['instruction'].tolist(),
     'prompt_ar': '',
     'response_en': subsampled_df['output'].tolist(),
     'response_ar': '',
     'source': dataset_repo,
    })], ignore_index=True)

df.shape

In [ ]:
from datasets import load_dataset

dataset_repo = 'HuggingFaceH4/Koala-test-set'
dataset = load_dataset(dataset_repo, split='test')

df_koala = pd.DataFrame(dataset)
df_koala.shape

In [ ]:
df_koala.head()

In [ ]:
df = pd.concat([df, pd.DataFrame(
    {'prompt_en': df_koala['prompt'].tolist(),
     'prompt_ar': '',
     'response_en': '',
     'response_ar': '',
     'source': dataset_repo,
    })], ignore_index=True)

df.shape

In [ ]:
from datasets import load_dataset

dataset_repo = 'HuggingFaceH4/ultrachat_200k'
dataset = load_dataset(dataset_repo, split='test_sft')

df_ultra = pd.DataFrame(dataset)
df_ultra.shape

In [ ]:
df_ultra.head()

In [ ]:
df_ultra.messages.iloc[0][1]

In [ ]:
subsampled_df = df_ultra[['prompt', 'messages']].sample(n=n_per_dataset).reset_index(drop=True)
collect_prompt = []
collect_response = []

for i in range(subsampled_df.shape[0]):
    messages = subsampled_df.messages.iloc[i]
    current_role = 'user'

    collect_prompt_sub = []
    collect_response_sub = []
    
    for message in messages:
        role = message['role']
        content = message['content']
        if role != current_role:
            break
        
        if current_role == 'user':
            collect_prompt_sub.append(content)
            current_role = 'assistant'
        else:
            collect_response_sub.append(content)
            current_role = 'user'

    if len(collect_prompt_sub) != 0:
        collect_prompt.append(collect_prompt_sub)
        collect_response.append(collect_response_sub)

len(collect_prompt), len(collect_response)

In [ ]:
df = pd.concat([df, pd.DataFrame(
    {'prompt_en': collect_prompt,
     'prompt_ar': '',
     'response_en': collect_response,
     'response_ar': '',
     'source': dataset_repo,
    })], ignore_index=True)

df.shape

In [ ]:
from datasets import load_dataset

dataset_repo = 'HuggingFaceH4/mt_bench_prompts'
dataset = load_dataset(dataset_repo, split='train')

df_mt = pd.DataFrame(dataset)
df_mt.shape

In [ ]:
df_mt.head()

In [ ]:
df = pd.concat([df, pd.DataFrame(
    {'prompt_en': df_mt['prompt'].tolist(),
     'prompt_ar': '',
     'response_en': '',
     'response_ar': '',
     'source': dataset_repo,
    })], ignore_index=True)

df.shape

In [ ]:
from datasets import load_dataset

dataset_repo = 'reciprocate/alpaca-eval'
dataset = load_dataset(dataset_repo, split='test')

df_alpaca = pd.DataFrame(dataset)
df_alpaca.shape

In [ ]:
df_alpaca.head()

In [ ]:
subsampled_df = df_alpaca[['prompt', 'selected']].sample(n=n_per_dataset).reset_index(drop=True)

subsampled_df['prompt'] = subsampled_df['prompt'].str.lstrip('USER: ')
subsampled_df['prompt'] = subsampled_df['prompt'].str.rstrip(' ASSISTANT: ')

In [ ]:
df = pd.concat([df, pd.DataFrame(
    {'prompt_en': [i for i in subsampled_df['prompt'].tolist()],
     'prompt_ar': '',
     'response_en': subsampled_df['selected'].tolist(),
     'response_ar': '',
     'source': dataset_repo,
    })], ignore_index=True)

df.shape

In [ ]:
from datasets import load_dataset

dataset_repo = 'natolambert/xstest-v2-copy'
dataset = load_dataset(dataset_repo, split='gpt4')

df_xstest = pd.DataFrame(dataset)
df_xstest.shape

In [ ]:
df_xstest.head()

In [ ]:
df_xstest.type.value_counts()

In [ ]:
df_xstest['focus_type'] = df_xstest.type.apply(lambda x: 'contrast' if 'contrast' in x else 'others')

In [ ]:
df_xstest.focus_type.value_counts()

In [ ]:
subsampled_df_contrast = df_xstest[df_xstest.focus_type == 'contrast']
subsampled_df_others = df_xstest[df_xstest.focus_type != 'contrast']

subsampled_df_contrast = subsampled_df_contrast[['prompt', 'completion']].sample(n=n_per_dataset//2).reset_index(drop=True)
subsampled_df_others = subsampled_df_others[['prompt', 'completion']].sample(n=n_per_dataset//2).reset_index(drop=True)

df = pd.concat([df, pd.DataFrame(
    {'prompt_en': subsampled_df_contrast['prompt'].tolist(),
     'prompt_ar': '',
     'response_en': subsampled_df_contrast['completion'].tolist(),
     'response_ar': '',
     'source': f'{dataset_repo}_contrast',
    })], ignore_index=True)

df = pd.concat([df, pd.DataFrame(
    {'prompt_en': subsampled_df_others['prompt'].tolist(),
     'prompt_ar': '',
     'response_en': subsampled_df_others['completion'].tolist(),
     'response_ar': '',
     'source': f'{dataset_repo}_others',
    })], ignore_index=True)

df.shape

In [ ]:
df_personal = pd.read_csv('red-team-personal.csv')
df_personal.shape

In [ ]:
df_personal.head()

In [ ]:
df = pd.concat([df, pd.DataFrame(
    {'prompt_en': df_personal['PROMPT'].tolist(),
     'prompt_ar': '',
     'response_en': '',
     'response_ar': '',
     'source': f'fanar_redteam',
    })], ignore_index=True)

In [ ]:
df.shape

In [ ]:
df.source.value_counts()

In [ ]:
df.to_csv('assets/analyzeDataset.csv', index=False)

### Translate content to arabic

In [ ]:
import pandas as pd
df = pd.read_csv('assets/analyzeDataset.csv')
df.shape

In [ ]:
df.source.value_counts()

In [ ]:
df.head()

In [ ]:
from utils import get_correct_format

df.prompt_en = df.prompt_en.apply(get_correct_format)
df.response_en = df.response_en.apply(get_correct_format)

In [ ]:
from utils import google_translator

google_translator(df.prompt_en.iloc[0])

In [ ]:
from tqdm import tqdm

cnt_exception = 0

for i in tqdm(range(df.shape[0])):
    prompt = df.prompt_en.iloc[i]
    try:
        if isinstance(prompt, list):
            prompt_tmp = []
            for p in prompt:
                prompt_tmp.append(google_translator(p))
            df.at[i, 'prompt_ar'] = prompt_tmp
        else:
            df.at[i, 'prompt_ar'] = google_translator(prompt)
    
        response = df.response_en.iloc[i]
        if isinstance(response, list):
            response_tmp = []
            for r in response:
                response_tmp.append(google_translator(r))
            df.at[i, 'response_ar'] = response_tmp
        else:
            df.at[i, 'response_ar'] = google_translator(response)
    except:
        cnt_exception += 1

    if not i%10:
        df.to_csv('assets/analyzeDataset.csv', index=False)

df.head()

In [ ]:
df.to_csv('assets/analyzeDataset.csv', index=False)

In [ ]:
df.head()

In [ ]:
i = 2

def select_random_rows(group):
    return group.sample(min(len(group), 5))

random_rows = df.groupby('source', group_keys=False).apply(select_random_rows)

random_rows.iloc[5*i:5*i+5]